In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 5,
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.10"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "source": [
        "# Pancreatic Cancer Detection with 3D CNN  \n",
        "\n",
        "**TCIA Pancreas-CT** 데이터셋을 활용해,\n",
        "1. 데이터 다운로드  \n",
        "2. 전처리  \n",
        "3. 3D CNN 모델 정의  \n",
        "4. 학습 및 평가  \n",
        "5. 결과 시각화  \n",
        "\n",
        "_Author: ChatGPT_\n"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 1. 라이브러리 설치 및 임포트  \n",
        "\n",
        "이 노트북은 `SimpleITK`, `requests`, `tensorflow` 등이 필요합니다."
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 필요한 패키지 설치 (한 번만 실행)\n",
        "!pip install SimpleITK requests tensorflow matplotlib\n"
      ],
      "execution_count": null
    },
    {
      "cell_type": "code",
      "source": [
        "import os\n",
        "import requests\n",
        "import SimpleITK as sitk\n",
        "import numpy as np\n",
        "import tensorflow as tf\n",
        "from tensorflow.keras import layers, models\n",
        "import matplotlib.pyplot as plt\n"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 2. TCIA Pancreas-CT 데이터 다운로드  \n",
        "TCIA REST API를 통해 첫 20명의 SeriesInstanceUID를 가져와 DICOM 파일을 저장합니다."
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "TCIA_API = \"https://services.cancerimagingarchive.net/services/v4/TCIA/query\"\n",
        "os.makedirs('data/dicom', exist_ok=True)\n",
        "\n",
        "# 1) 환자 Study 목록 조회\n",
        "resp = requests.get(f\"{TCIA_API}/getPatientStudy\",\n",
        "                    params={\"Collection\":\"Pancreas-CT\",\"Modality\":\"CT\",\"format\":\"json\"})\n",
        "studies = resp.json()\n",
        "\n",
        "# 2) SeriesInstanceUID 목록 수집 (최대 20개)\n",
        "series_uids = []\n",
        "for s in studies:\n",
        "    sid = s['StudyInstanceUID']\n",
        "    series = requests.get(f\"{TCIA_API}/getSeries\",\n",
        "                          params={\"StudyInstanceUID\":sid, \"format\":\"json\"}).json()\n",
        "    for ser in series:\n",
        "        if ser['Modality']=='CT':\n",
        "            series_uids.append(ser['SeriesInstanceUID'])\n",
        "    if len(series_uids)>=20:\n",
        "        break\n",
        "\n",
        "print(f\"Collected {len(series_uids)} series\")\n",
        "\n",
        "# 3) 각 Series 다운로드 (예: simple-series-downloader 스크립트 사용)\n",
        "# ※ 실제 다운로드는 GitHub https://github.com/CancerImagingArchive/Simple-Series-Downloader 참고\n"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 3. DICOM → 3D 볼륨 전처리 함수  \n",
        "- 슬라이스 재샘플링 → (64, 128, 128) 크기의 NumPy 배열\n",
        "- HU 윈도우, 0–1 정규화"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "def load_and_preprocess(dicom_folder,\n",
        "                         target_size=(64,128,128),\n",
        "                         window=(-75, 175)):\n",
        "    # 1) 시리즈 로드\n",
        "    reader = sitk.ImageSeriesReader()\n",
        "    fns = reader.GetGDCMSeriesFileNames(dicom_folder)\n",
        "    reader.SetFileNames(fns)\n",
        "    img = reader.Execute()\n",
        "    # 2) 윈도우 설정 (level=-75, width=250)\n",
        "    img = sitk.IntensityWindowing(img,\n",
        "                                  windowMinimum=window[0],\n",
        "                                  windowMaximum=window[1],\n",
        "                                  outputMinimum=0.0,\n",
        "                                  outputMaximum=1.0)\n",
        "    # 3) 재샘플링\n",
        "    resampled = sitk.Resample(\n",
        "        img,\n",
        "        [target_size[2], target_size[1], target_size[0]],  # (W,H,D)\n",
        "        sitk.Transform(),\n",
        "        sitk.sitkLinear,\n",
        "        img.GetOrigin(),\n",
        "        [1.0,1.0,3.0],\n",
        "        img.GetDirection(),\n",
        "        0,\n",
        "        img.GetPixelID()\n",
        "    )\n",
        "    arr = sitk.GetArrayFromImage(resampled)  # shape: (D,H,W)\n",
        "    return arr[..., np.newaxis]           # (D,H,W,1)\n"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 4. 3D CNN 모델 정의  "
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "def build_3dcnn(input_shape=(64,128,128,1)):\n",
        "    inp = layers.Input(input_shape)\n",
        "    x = layers.Conv3D(32,3,padding='same',activation='relu')(inp)\n",
        "    x = layers.Conv3D(32,3,padding='same',activation='relu')(x)\n",
        "    x = layers.MaxPool3D(2)(x)\n",
        "    x = layers.Conv3D(64,3,padding='same',activation='relu')(x)\n",
        "    x = layers.MaxPool3D(2)(x)\n",
        "    x = layers.Conv3D(128,3,padding='same',activation='relu')(x)\n",
        "    x = layers.MaxPool3D(2)(x)\n",
        "    x = layers.Conv3D(256,3,padding='same',activation='relu')(x)\n",
        "    x = layers.Flatten()(x)\n",
        "    x = layers.Dense(512,activation='relu')(x)\n",
        "    out = layers.Dense(1,activation='sigmoid')(x)\n",
        "    model = models.Model(inp, out)\n",
        "    return model\n",
        "\n",
        "model = build_3dcnn()\n",
        "model.compile(optimizer='adam',\n",
        "              loss='binary_crossentropy',\n",
        "              metrics=['accuracy'])\n",
        "model.summary()"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 5. 학습용 데이터 준비  \n",
        "- 앞 10개 series: PDAC(레이블=1), 뒤 10개 series: 정상(레이블=0)"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 예시: 메모리에 바로 로드\n",
        "X, y = [], []\n",
        "for i, sid in enumerate(series_uids[:20]):\n",
        "    folder = f\"data/dicom/{sid}\"\n",
        "    vol = load_and_preprocess(folder)\n",
        "    X.append(vol)\n",
        "    y.append(1 if i<10 else 0)\n",
        "X = np.stack(X, axis=0)\n",
        "y = np.array(y)\n",
        "print(X.shape, y.shape)"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 6. 학습 및 검증  "
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "history = model.fit(\n",
        "    X, y,\n",
        "    batch_size=4,\n",
        "    epochs=20,\n",
        "    validation_split=0.2\n",
        ")"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 7. 학습 결과 시각화  "
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "plt.figure(figsize=(6,4))\n",
        "plt.plot(history.history['loss'], label='train loss')\n",
        "plt.plot(history.history['val_loss'], label='val loss')\n",
        "plt.xlabel('Epoch')\n",
        "plt.ylabel('Loss')\n",
        "plt.legend()\n",
        "plt.title('Training vs Validation Loss')\n",
        "plt.show()\n",
        "\n",
        "plt.figure(figsize=(6,4))\n",
        "plt.plot(history.history['accuracy'], label='train acc')\n",
        "plt.plot(history.history['val_accuracy'], label='val acc')\n",
        "plt.xlabel('Epoch')\n",
        "plt.ylabel('Accuracy')\n",
        "plt.legend()\n",
        "plt.title('Training vs Validation Accuracy')\n",
        "plt.show()"
      ],
      "execution_count": null
    },
    {
      "cell_type": "markdown",
      "source": [
        "---  \n",
        "_이 노트북(.ipynb)을 GitHub에 업로드하신 뒤, 특히 `data/` 폴더에 DICOM이 잘 들어있는지 확인하면, 누구나 클론 후 `jupyter notebook` 커맨드로 바로 실행해 볼 수 있습니다._"
      ]
    }
  ]
}
